<a href="https://colab.research.google.com/github/KevinW824/CSE-144-Applied-Machine-Learning/blob/main/Summer_21_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# Assignment 5

**DUE: Sunday July 24, 2021 11:59pm**

Turn in the assignment via Canvas.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. First, restart the kernel (in the menubar, select Runtime→→Restart runtime) and then run all cells (in the menubar, select Runtime→→Run All).

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
NAME = "Bowen Wang"
STUDENT_ID = "bwang93"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

## Question 1 Bitcoin price Prediction

Bitcoin, amongst other assets like AMC and GME, has been all the rage this past year and reached an all time high of \$63,729.5 per bitcoin. Since reaching it's high, the price has dropped by nearly a half. Analysts continue to feed the frenzy by releasing price predictions that range from from \$500,000 to \$9,000 per bitcoin in the next year. You will make an Recurrent neural network model to gain some insight into price prediction. Yahoo! Finance is a trusted name in free financial information and has been with us since the internet's early years. You'll be using data gathered obtained from https://finance.yahoo.com/quote/BTC-USD/history/ to train your recurrent neural network.

Run the following code cell to download the training and test data. It might take a while to download the zip file and extract it.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
import zipfile
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1JbrxeSqazKn-WjizVDPG9VvhkXVG--D-'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('BTC-USD.csv')


In [ ]:
# Create pandas dataframe
data = pd.read_csv('BTC-USD.csv')

In [ ]:
# Plot data.head()
data.head()

Let's plot the bitcoin price. First, we will make a plot of bitcoin price vs the days after September 15, 2014, the start date of this dataset. Day "0" indicates September 15, 2014.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns # Used for plotting

g = sns.lineplot(x = np.linspace(1,2495,2495), y = data['High'].values.reshape(-1))


Next, we plot bitcoin vs. days. But this time there are dates added to the graph.

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns # Used for plotting

g = sns.lineplot(x = np.linspace(1,2495,2495), y = data['High'].values.reshape(-1))
#g.map(plt.plot, "a", "v", marker="o")
g.set(xticks=np.arange(0,2500,200))
g.set_xticklabels(rotation=30, labels = data['Date'][0::200])

### Part a) Data preprocessing (5 points)

In this section you will preprocess the in order to train a recurrent neural network. We can see that there 5 columns, "Date", "Open", "High", "Low", "Close", "Adjusted Close", and "Volume". We will only use the High column.

In [ ]:
# Create a dataframe that only contains High Column.
# Hint: it may be helpful to now cast your pruned dataframe to a numpy array.

data_high = ### YOUR CODE HERE ###



Recall [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) from sklearn. Use it to scale the data for our analysis. 

In [ ]:
### YOUR CODE HERE ###
scaler = ### YOUR CODE HERE ###

data_normalized = ### YOUR CODE HERE ###


You will implement a simple autoregressive recurrent neural network using the standard tensorflow RNN architectures. An autoregressive model originates from the literature on time-series models where observations from the previous time-steps are used to predict the value at the current time step. To implemement an autoregressive model, we will simply augment the data so that "time_steps" number of previous days are fed to the model at the current time step in order to form a prediction. The function to augment the data is given to you:

In [ ]:

def create_dataset(dataset, time_steps=1):
    """
    Generate a dataset where the time series X[n] contains the readings for the 'time_step' previous days 
    and y contains the price for today.
    args:
    dataset: numpy array, the data
    time_steps: the number of previous days to feed to RNN

    returns:
    tuple: a dataset with x[i] containing 'time_step' number of previous prices, target price for x[i]
    """
    dataX, dataY = [],[]
    for i in range(len(dataset)-time_steps-1):
        a = dataset[i:(i+time_steps)]
        dataX.append(a)
        dataY.append(dataset[i + time_steps])
    return np.array(dataX), np.array(dataY)

In [ ]:
# Choose the number of time steps that the model "looks back"
time_steps = ### YOUR CODE HERE ###

# Produce your dataset based on the number of days the model could look back

### YOUR CODE HERE ###




In [ ]:
# Check the shape of your dataset; should be (2495-time_steps-1, time_steps) and (2495-time_steps-1,)
print(X.shape, y.shape)

### Part b) Data Partitioning (5 points)

Split data into train and test sets. Use 80\% for training and 20\% for testing. **Note**: you need to split the data in time (the begining 80\% of the days from start date will be the training data and the remaining 20\% will be test data).

In [ ]:
### YOUR CODE HERE ###


For this dataset, you need to reshape the partitions for the model to be able to process them.

In [ ]:
# Reshape input to be [samples, time steps, features].
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

### Part c) RNN Model (20 points)
---

In this part you will create a model using an RNN layer (LSTM or GRU, unidirectional or bidirectional) and train it on your training data. You will also plot training and validation loss. Use mean squared error as your model's metric.

Compile your model and display the summary:

In [ ]:
# Build your model

### YOUR CODE HERE ###

loss = ### YOUR CODE HERE ###

opt = ### YOUR CODE HERE ###

metrics = ### YOUR CODE HERE ###



In [ ]:

batchsize = ### YOUR CODE HERE ###

epochs =  ### YOUR CODE HERE ###

# Fit model
history = ### YOUR CODE HERE ###


In [ ]:
# Plot the Model loss

### YOUR CODE HERE ###


### Part d) More Advanced RNN Model (20 points)
---
In this part you will create an RNN model with the number of layers and architerure you prefer. Train it on your training data. You will also plot training and validation loss. Again, use mean squared error as your metric. In this part, you can try different models and use different hyper-parameters and report only the best one.

Compile your model and display the summary:

In [ ]:
# Build your model

### YOUR CODE HERE ###

loss = ### YOUR CODE HERE ###

opt = ### YOUR CODE HERE ###

metrics = ### YOUR CODE HERE ###


In [ ]:
batchsize = ### YOUR CODE HERE ###
epochs =  ### YOUR CODE HERE ###

# Fit model
history =  ### YOUR CODE HERE ###

In [ ]:
# Plot the Model loss

### YOUR CODE HERE ###

### Part e): Looking at the Predictions (10 points)
---

Now, Using the final (best) model you trained, show your model's performance on the test set. Plot the model's prediction for Bitcoin Price along with the actual test set prices. Lastly, note how your model's predictions change with your model's architecture and the number of days you "look back". Does your model perform better with more "look back days" or less. Did adding more layers help? Does your model use dropout or batchnormalization?

**Note:** Your model is trained on normalized data. Inorder to transform your model's predictions to the original price range you will likely need to use sklearn's inverse_transform (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
### YOUR CODE HERE ###

[YOUR ANSWER HERE]

## Question 3: Denoising Autoencoder


You will now build a simple denoising autoencoder using the MNIST Fashion dataset. Begin by getting our imports and downloading the dataset.

In [ ]:
import os
import numpy as np
from keras.datasets import fashion_mnist

# Data visualizaton.
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import random as rn
rn.seed(123)
np.random.seed(seed=123)

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
%matplotlib inline

In [ ]:
# Plot some of the images from x_test
fig, ax = plt.subplots(5, 4)
fig.set_size_inches(15, 15)

for i in range(5):
    for j in range(4):
        l=rn.randint(0, len(y_test))
        ax[i, j].imshow(x_test[l], cmap='gray')
        ax[i, j].set_title('Label: ' + str(y_test[l]))
        # Hide grid lines
        ax[i, j].grid(False)
        # Hide axes ticks
        ax[i, j].set_xticks([])
        ax[i, j].set_yticks([])
        
plt.tight_layout()

As expected, we have the MNIST Fashion dataset. Now, noise will be added to the images in boath x_train and x_test. Then, we will plot some of the noisy images from x_train. This may take a little while.

In [ ]:
def corrupt_pixel(image):
    """selects a certain number of pixels (between 50 and 100) and changes their value to 255"""
    num_pixels = np.random.randint(50,100)
    img = image.copy()

    for i in range(num_pixels):
        cell = np.random.randint(0,28,size = 2)
        img[cell[0]][cell[1]] = np.random.randint(125,255)
    return img

In [ ]:
# corrupt the images
x_train_noisy = [] #x_train with noise added
for image in x_train:
    x_train_noisy.append(corrupt_pixel(image))

x_test_noisy = [] #x_test with noise added
for image in x_test:
    x_test_noisy.append(corrupt_pixel(image))

In [ ]:
# Replot some of the test images with noise added
rn.seed(123)
np.random.seed(seed=123)

fig, ax = plt.subplots(5, 4)
fig.set_size_inches(15, 15)

for i in range(5):
    for j in range(4):
        l=rn.randint(0, len(y_test))
        ax[i, j].imshow(x_test_noisy[l], cmap='gray')
        ax[i, j].set_title('Label: ' + str(y_test[l]))
        # Hide grid lines
        ax[i, j].grid(False)
        # Hide axes ticks
        ax[i, j].set_xticks([])
        ax[i, j].set_yticks([])
        
plt.tight_layout()

We can now see that we have corrupted the images. We will now build a denoising autoencoder to hopefully remove some of the noise. First, data will be normalize the data and an extra dimension will be added to the images.

In [ ]:
# Scale X_train to 255. which is max
x_train  = np.array(x_train)/255.

# Reshape X_train to (N,28,28,1) for convolution layers
x_train = x_train.reshape(len(x_train),28,28,1)

# Scale X_test to 255. which is max
x_test  = np.array(x_test)/255.

# Reshape X_train to (N,28,28,1) for convolution layers
x_test = x_test.reshape(len(x_test),28,28,1)

# Scale X_train to 255. which is max
x_train_noisy  = np.array(x_train_noisy)/255.

# Reshape X_train to (N,28,28,1) for convolution layers
x_train_noisy = x_train_noisy.reshape(len(x_train_noisy),28,28,1)

# Scale X_test to 255. which is max
x_test_noisy  = np.array(x_test_noisy)/255.

# Reshape X_train to (N,28,28,1) for convolution layers
x_test_noisy = x_test_noisy.reshape(len(x_test_noisy),28,28,1)

### Part a) Build the autoencoder (20 points)


 Now we will build the autoencoder. You are free to build an architecture of your choice. Again, You are welcome to use any code from previous class exercises, section handouts, and lectures. Build your model below and comment on your model's architecture. Do you have convolution layers, or a fully connected encoder, dropout and/or batch normalization, what kind of activation did you use? It may help to think about the encoder/decoder architectures simultaneously.

You may use the TensorFlow documentation freely. You might also find online tutorials helpful. However, all code that you submit must be your own.


In [ ]:
# Build your model

### YOUR CODE HERE ###

[YOUR ANSWER HERE]

### Part b) Training the autoencoder (10 points)
Write code that trains your autoencoder. Use mean square error as your loss, choose an optimizer of your choice. Be

Plot the training curve as well.

In [ ]:
# Compile your model

### YOUR CODE HERE ###

# Train the autoencoder

### YOUR CODE HERE ###


In [ ]:
# Plot the training curve for loss

### YOUR CODE HERE ###


### Part c) Plot images and comment (10 points)

Plot images some of the "denoised" images from the test set. Comment on how your model has performed. Was your model able to denoise the image successfully? Is there anything unexpected results about the images? MSE error is known to smooth an image too much, did your model suffere the same result?

In [ ]:
# Plot images some of the "denoised" images from the test set

### YOUR CODE HERE ###


[YOUR ANSWER HERE]